## 필요한 모듈들 임포트

## Fine tuning
사전 훈련된 모델을 기반으로 아키텍쳐를 새로운 목적에 맞게 변형하고 이미 학습된 모델의 가중치를 미세하게 조정해 학습시키는 방법
    - 사전 훈련된 모델의 가중치를 초기값으로 사용하고 추가로 학습
    - CNN 베이스의 사전학습 모델을 사용할 때에는 이전에 학습한 내용들을 모두 잊어버릴 위험이 있기 때문에 작은 learning rate를 사용하는것이 바람직함
[방법1] 모델 전체를 새로 학습
- 사전 학습 모델의 구조만 사용하면서, 자신의 데이터셋에 맞게 모델을 전부 새롭게 학습시키는 방법
- 데이터의 크기가 크고 유사성이 작을 때 사용
[방법2] Conv base는 일부분 고정(Freezing)하고 나머지 Conv Base 계층과 Classifier를 새롭게 학습
- Conv base: 합성곱 층과 풀링 층이 여러 겹 쌓여있는 부분으로 특징을 추출하는 역할
- Classifier: 주로 완전연결계층으로 구성되며 Conv base가 추출한 특징들을 잘 학습해 각각의 샘플들을 알맞은 class로 분류
- 낮은 레벨의 계층은 일반적이고 독리적인 특징(신형성)을 추출하고, 높은 레벨의 계층은 구체적이고 명확한 특징(형상)을 추출
- 크기가  크고 유사성도 높은 데이터셋일 때
[방법3] Conv Base는 고정하고 Classifier만 새로 학습
- 컴퓨팅 연산 능력이 부족하거나, 데이터셋이 너무 작을 때, 혹은 적용하려는 task와 사전학습에 쓰인 데이터가 매우 비슷할 때 사용

[크기가 작고 유사성도 작은 데이터]
- 방법2를 쓰되 조금 더 깊은 계층까지 새로 학습시키
- Data Augmentation 하기

[Feature Extraction]
사전 훈련된 모델의 하위 층을 동결하고, 상위 층을 새로운 작업을 위해 수정하지 않고 사용하는 것 -> 데이터분류기(마지막 완전연결층) 부분만 새로 만드는 것
    - 사전 훈련된 모델(고정): 중요한 특성 추출(학습 X)
    - 데이터분류기: 추출된 특성을 입력받아 최종적으로 이미지에 대한 클래스를 분류(학습O)
    - 가능한 모델: Xception, Inception V3, ResNet50, VGG16, VGG19, MobileNet

[ 분류기 수정하는 방법 ]
Fine tunning을 하기 전 input값으로 받는 이미지의 크기는 무조건 확인해야함
    - 대부분의 모델은 (224, 224)이지만 inception_v3의 경우 (299, 299)
1. print(model)로 마지막 층의 in_features 확인
2. nn.Linear(in_features, num_classes)로 클래스 수 변경
    - 예시1: Resnet
        - (fc): Linear(in_features=512, out_features=1000, bias=True)
            - => model.fc = nn.Linear(512, num_classes)
    - 예시2: Alexnet
        - (classifier): Sequential(... (6): Linear(in_features=4096, out_features=1000, bias=True)
            - => model.classifier[6] = nn.Linear(4096, num_classes)
    - 예시3(특이구조): Squeezenet
        - - output은 classifier의 첫번째 레이어인 1X1 convolution layer로부터 나옴
        - (classifier): Sequential((0): Dropout(p=0.5)  (1): Conv2d(512, 1000, kernel_size=13, stride=1, padding=0)  (2): ReLU(inplace)  (3): AvgPool2d(kernel_size=13, stride=1, padding=0))
            -  => model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
    - 예시4(특이구조): Inception v3
        - Rethinking을 하기 때문에 학습과정에서 output이 두개의 레이어로부터 나옴
        - 두번째 output(auxiliary output): AuxLogits 부분을 포함하는 네트워크에 포함됨
        - 주된 output은 네트워크의 마지막 레이어에서 출력됨
            -  test 시에는 이 output만 고려함
        - (AuxLogits): InceptionAux(... (fc): Linear(in_feature=768, out_features=1000, bias=True) ... (fc): Linear(in_features=2048, out_features=1000, bias=True)
            -  finetune 하기 위해서는 두 레이어를 reshape 해줘야함
            -  => model.AuxLogits.fc = nn.Linear(768, num_classes)
            -  model.fc = nn.Linear(2048, nuum_classes)

[ 사용가능한 함수들 ]
- get_model(name, **config): 모델 이름과 환경설정을 인수로 받아 해당 모델의 인스턴스를 반환
    - get_model("quantized_mobilenet_v3_large", weights="DEFAULT")
- get_model_weight(name): 해당 모델의 열거형 가중치 클래스 반환
    - 예1: get_model_weights("quantized_mobilenet_v3_large")
    - 예2: get_model_weights(torchvision.models.quantization.mobilenet_v3_large)
    - 예1 = 예2
- get_weight(name): 열거형 가중치 변수의 이름으로 값을 가져옴
    - 예: get_weight("MobileNet_V3_Large_QuantizedWeights.DEFAULT")
- list_models(\[module]): 해당 이름으로 등록된 모델 목록을 반환


In [ ]:
import copy
import time
from tqdm.notebook import tqdm as tqdm_notebook
import cv2
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import random_split, DataLoader

from custom_models import CustomMobileNetV3Large, QuantizedCustomMobileNetV3Large, CustomMnasNet1_3, QuantizedCustomMnasNet1_3

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# 추론코드

img = cv2.imread('apple.jpg')
img = transform_test(img).unsqueeze(0)

with torch.no_grad():
    outputs = model(img.to(device))

_, predicted_class = torch.max(outputs, 1)
predicted_class = predicted_class.item()
predicted_class

model_list = ['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception', 'mobilenet']

## Helper Functions

In [ ]:
# 모델학습 헬퍼 함수
def train_models(model, dataloaders, criterion, optimizer, num_epochs=10, is_inception=False, quantize=False):
    torch.cuda.empty_cache()
    since = time.time()

    val_acc_history = []
    val_top5_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    patience = 3
    no_improvement_count = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1} / {num_epochs}')
        print('-' * 10)

        # 각 에폭은 학습/검증 phase를 가짐
        for phase in ['train', 'val']:
            print(f'>>>>> phase: {phase}<<<<<')
            if phase == 'train':
                model.train()  # 트레이닝 모드 설정
            else:
                if quantize == True:
                    # 모델을 양자화 + 추론모드
                    model = torch.quantization.convert(model.eval(), inplace=False)
                else:
                    model.eval()  # 추론 모드 설정

            running_loss = 0.0
            running_corrects = 0
            running_top5_corrects = 0

            # 데이터 학습하기
            for inputs, labels in tqdm_notebook(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 파라미터 기울기 초기화
                optimizer.zero_grad()

                # forward
                # 학습 모드인 경우에만 history 추적
                with torch.set_grad_enabled(phase == 'train'):
                    # 모델의 ouptut과 loss를 구함
                    # inception의 경우 학습시 auxiliary output이 있는 특수 케이스임.
                    #   학습시: final output과 auxiliary output을 더하는 과정이 필요함
                    #   테스트시: final output만 고려
                    # Auxilary output을 같이 고려해야하는 학습단계
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4 * loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # top-5 정확도 계산
                    _,top5_preds = torch.topk(outputs, 5)
                    top5_corrects = torch.sum(top5_preds == labels.view(-1, 1))

                    # 학습(backward + optimize)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (epoch + 1) % 5 == 0:
                            # 모델 및 다른 정보 저장
                            save_path = f'save/{type(model).__name__}_epoch{epoch+1}_quntize({quantize}).pth'
                            torch.save({
                                'model_state_dict': model.state_dict(),  # 모델 가중치 저장
                                'optimizer_state_dict': optimizer.state_dict(),  # 옵티마이저 상태 저장 (선택적)
                                'epoch': epoch+1,  # 현재 학습 에폭 저장 (선택적)
                                # 다른 필요한 정보 저장 (선택적)
                            }, save_path)

                # loss 구하기
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_top5_corrects += top5_corrects

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            epoch_top5_acc = running_top5_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # 모델 깊은복사
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                no_improvement_count = 0  # 개선이 있었으므로 카운트 초기화
            if phase == 'val':
                val_acc_history.append(epoch_acc.item())
                val_top5_acc_history.append(epoch_top5_acc.item())
                no_improvement_count += 1  # 개선이 없었으므로 카운트

            # Early Stopping 확인
            if no_improvement_count >= patience:
                print(f"No improvement in validation accuracy for {patience} epochs. Early stopping...")
                break  # 학습 종료

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.4f}'.format(best_acc))
    print('Best top5 val Acc: {:.4f}'.format(epoch_top5_acc))

    # 베스트 모델 가중치를 로드
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, val_top5_acc_history


def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


#### 모델 파라미터의 requires_grad 속성
- 기본적으로 사전  훈련된 모델을 로드할 때 모든 매개변수의 requires_grad = True로 설정되어 있음
    - True: 처음부터 훈련하거나 fine tuning할 때는 True
    - False(Freeze): 기존 layer의 weight를 고정

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True, quantize=False):
    # 모델마다 다르게 지정될 변수들 초기화
    model_ft = None
    input_size = 0

    if model_name == 'resnet':
        if quantize:
            '''Quantized Resnet50'''
            weights = models.quantization.ResNet50_QuantizedWeights.DEFAULT
            model_ft = models.quantization.resnet50(weights=weights, quantize=True)
        else:
            '''Resnet50'''
            model_ft = models.resnet50(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        set_parameter_requires_grad(model_ft, feature_extract)
        input_size = 224

    elif model_name == 'mobilenet':
        if quantize:
            '''Quantized Mobilenet v3 large'''
            model_ft = QuantizedCustomMobileNetV3Large(num_classes=500).to(device)

            # 양자화 설정 - gbgemm 백엔드 사용
            backend = 'fbgemm'
            # 양자화 스키마 설정 (예: symmetric)
            quantization_params = torch.quantization.get_default_qconfig(backend)
            quantization_params = torch.quantization.QConfig(activation=quantization_params.activation, weight=quantization_params.weight)

            # 양자화 준비
            model.qconfig = quantization_params
            model_ft= torch.quantization.prepare_qat(model_ft, inplace=False)

        else:
            model_ft = CustomMobileNetV3Large(num_classes=500).to(device)
        set_parameter_requires_grad(model_ft, feature_extract)
        input_size = 224

    elif model_name == 'mnasnet':
        if quantize:
            '''Quantized Mnasnet 1_3'''
            model_ft = QuantizedCustomMnasNet1_3(num_classes=500).to(device)

            # 양자화 설정 - gbgemm 백엔드 사용
            backend = 'fbgemm'
            model_ft.qconfig = torch.quantization.get_default_qat_qconfig(backend)

            # 양자화 준비
            model_ft= torch.quantization.prepare_qat(model_ft, inplace=False)

        else:
            model_ft = models.mnasnet1_3(weights=models.MNASNet1_3_Weights.IMAGENET1K_V1)
            num_ftrs = model_ft.classifier[1].in_features
            model_ft.classifier[1] = nn.Linear(in_features=1280, out_features=num_classes, bias=True)
            # model_ft = CustomMnasNet1_3(num_classes=500).to(device)
        set_parameter_requires_grad(model_ft, feature_extract)
        input_size = 224

    elif model_name == 'efficientnet':
        if quantize:
            pass
        else:
            model_ft = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
            num_ftrs = model_ft.classifier[1].in_features
            model_ft.classifier[1] = nn.Linear(num_ftrs, num_classes, bias=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        input_size = 224

    elif model_name == 'shufflenet':
        if quantize:
            pass
        else:
            model_ft = models.shufflenet_v2_x2_0(weights=models.ShuffleNet_V2_X2_0_Weights.IMAGENET1K_V1)
            num_ftrs = model_ft.fc.in_features
            model_ft.fc = nn.Linear(num_ftrs, num_classes, bias=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        input_size = 224

    elif model_name == 'alexnet':
        '''AlexNet'''
        model_ft = models.alexnet(pretrianed=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_featrues
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == 'vgg':
        '''VGG11_bn'''
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == 'squeezenet':
        '''Squeezenet 1.0'''
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == 'densenet':
        '''Densenet 121'''
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == 'inception':
        '''Inception V3'''
        if quantize:
            weights = models.quantization.Inception_V3_QuantizedWeights.DEFAULT
            model_ft = models.quantization.inception_v3(weights=weights, quantize=True)
        else:
            model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 299  # 다른 모델과 다르게 299 사이즈를 사용
    else:
        print('모델의 이름을 잘못 입력하여 종료합니다...')
        exit()

    return model_ft.to(device), input_size

## 모델 초기화

In [ ]:
# 데이터 경로
data_root = f'D:/data/training/sources/cropped'

# 데이터 변환
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ImageFolder로 전체 데이터셋 생성
dataset = datasets.ImageFolder(root=data_root, transform=data_transforms)

# 데이터셋 분할 (예: 80% 훈련, 20% 유효성 검사)
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size

train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])
valid_dataset, test_dataset = random_split(valid_dataset, [0.95, 0.05])

# 데이터 로더 생성
batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# dataloaders_dict = {'train': train_dataloader, 'val': valid_dataloader, 'test': test_dataloader}
# dataloaders_dict = {'train': valid_dataloader, 'val': test_dataloader}  # 성능 빠르게 확인용
dataloaders_dict = {'train': test_dataloader, 'val': test_dataloader}  # 코드 돌아가는지 확인용

print('train > ', len(train_dataloader), '\nval > ', len(valid_dataloader), '\ntest > ', len(test_dataloader))

In [ ]:
def testing_models(dataloaders_dict, num_epochs, num_classes=1000, feature_extract=False, use_pretrained=True, quantize=False):
    losses = {}

    for model_name in ['mobilenet', 'mnasnet', 'efficientnet', 'shufflenet']:
        torch.cuda.empty_cache()

        model, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained, quantize=quantize)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.NAdam(model.parameters(), lr=0.001)

        print('device > ', device)
        print(f'{model_name} / pretrained({use_pretrained}) / quantize({quantize})')

        model, hist, top5_hist = train_models(model, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs, is_inception=False, quantize=quantize)

        losses[model_name] = {
            'top1': hist,
            'top5': top5_hist
        }

    return losses

In [ ]:
losses = testing_models(dataloaders_dict, num_epochs=5, num_classes=500, feature_extract=False, use_pretrained=True, quantize=False)

print(losses)

In [ ]:
torch.cuda.empty_cache()

# mobilenet, mnasnet, efficientnet, shufflenet
model_name = 'efficientnet'
num_classes = 500
feature_extract = False
use_pretrained = True
quantize = False

model_efficientnet, input_size_efficientnet = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained, quantize=quantize)

criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(model_efficientnet.parameters(), lr=0.001)

print('device > ', device)
print(f'{model_name} / pretrained({use_pretrained}) / quantize({quantize})')
num_epochs = 5
model_efficientnet, hist_efficientnet, top5_hist_efficientnet = train_models(model_efficientnet, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs, is_inception=False, quantize=quantize)

In [ ]:
# 모델 및 다른 정보 저장
save_path = f'save/{model_name}_epoch{num_epochs}_quntize({quantize}).pth'
torch.save({
    'model_state_dict': model_efficientnet.state_dict(),  # 모델 가중치 저장
    'optimizer_state_dict': optimizer.state_dict(),  # 옵티마이저 상태 저장 (선택적)
    'epoch': num_epochs,  # 현재 학습 에폭 저장 (선택적)
    # 다른 필요한 정보 저장 (선택적)
}, save_path)

In [ ]:
torch.cuda.empty_cache()

# mobilenet, mnasnet, efficientnet, shufflenet
model_name = 'shufflenet'
num_classes = 500
feature_extract = False
use_pretrained = True
quantize = False

model_shufflenet, input_size_shufflenet = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained, quantize=quantize)

criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(model_shufflenet.parameters(), lr=0.001)

print('device > ', device)
print(f'{model_name} / pretrained({use_pretrained}) / quantize({quantize})')
num_epochs = 5
model_shufflenet, hist_shufflenet, top5_hist_shufflenet = train_models(model_shufflenet, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs, is_inception=False, quantize=quantize)

# 모델 및 다른 정보 저장
save_path = f'save/{model_name}_epoch{num_epochs}_quntize({quantize}).pth'
torch.save({
    'model_state_dict': model_shufflenet.state_dict(),  # 모델 가중치 저장
    'optimizer_state_dict': optimizer.state_dict(),  # 옵티마이저 상태 저장 (선택적)
    'epoch': num_epochs,  # 현재 학습 에폭 저장 (선택적)
    # 다른 필요한 정보 저장 (선택적)
}, save_path)

In [ ]:
torch.cuda.empty_cache()

# mobilenet, mnasnet, efficientnet, shufflenet
model_name = 'mobilenet'
num_classes = 500
feature_extract = False
use_pretrained = True
quantize = False

model_mobilenet, input_size_mobilenet = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained, quantize=quantize)
model_mobilenet = model_mobilenet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(model_mobilenet.parameters(), lr=0.001)

print('device > ', device)
print(f'{model_name} / pretrained({use_pretrained}) / quantize({quantize})')
num_epochs = 5
model_mobilenet, hist_mobilenet, top5_hist_mobilenet = train_models(model_mobilenet, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs, is_inception=False, quantize=quantize)

# 모델 및 다른 정보 저장
save_path = f'save/{model_name}_epoch{num_epochs}_quntize({quantize}).pth'
torch.save({
    'model_state_dict': model_mobilenet.state_dict(),  # 모델 가중치 저장
    'optimizer_state_dict': optimizer.state_dict(),  # 옵티마이저 상태 저장 (선택적)
    'epoch': num_epochs,  # 현재 학습 에폭 저장 (선택적)
    # 다른 필요한 정보 저장 (선택적)
}, save_path)

In [ ]:
torch.cuda.empty_cache()

# mobilenet, mnasnet, efficientnet, shufflenet
model_name = 'mnasnet'
num_classes = 500
feature_extract = False
use_pretrained = True
quantize = False

model_mnasnet, input_size_mnasnet = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained, quantize=quantize)
model_mnasnet = model_mnasnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.NAdam(model_mnasnet.parameters(), lr=0.001)

print('device > ', device)
print(f'{model_name} / pretrained({use_pretrained}) / quantize({quantize})')
num_epochs = 5
model_mnasnet, hist_mnasnet, top5_hist_mnasnet = train_models(model_mnasnet, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs, is_inception=False, quantize=quantize)

# 모델 및 다른 정보 저장
save_path = f'save/{model_name}_epoch{num_epochs}_quntize({quantize}).pth'
torch.save({
    'model_state_dict': model_mnasnet.state_dict(),  # 모델 가중치 저장
    'optimizer_state_dict': optimizer.state_dict(),  # 옵티마이저 상태 저장 (선택적)
    'epoch': num_epochs,  # 현재 학습 에폭 저장 (선택적)
    # 다른 필요한 정보 저장 (선택적)
}, save_path)

In [ ]:
# losses = {
#     'efficientnet': {
#         'top1': hist_efficientnet,
#         'top5': top5_hist_efficientnet
#     },
#     'shufflenet': {
#         'top1': hist_shufflenet,
#         'top5': top5_hist_shufflenet
#     },
#     'mobilenet': {
#         'top1': hist_mobilenet,
#         'top5': top5_hist_mobilenet
#     },
#     'mnasnet': {
#         'top1': hist_mnasnet,
#         'top5': top5_hist_mnasnet
#     },
# }
losses = {
    'efficientnet': {
        'top1': hist_efficientnet,
        'top5': top5_hist_efficientnet
    }}

# 딕셔너리 내의 PyTorch Tensor를 .item()을 사용하여 Python 숫자로 변환
def convert_to_items(d):
    for key, value in d.items():
        if isinstance(value, dict):
            d[key] = convert_to_items(value)
        elif isinstance(value, list):
            d[key] = [item.item() if isinstance(item, torch.Tensor) else item for item in value]
    return d

data_dict_items = convert_to_items(losses)

import json

with open('save/losses.json', 'w') as f:
    json.dump(losses, f)

# torch.save(hist_efficientnet, 'save/efficientnet_hist.pt')
# torch.save(top5_hist_efficientnet, 'save/efficientnet_hist_top5.pt')

In [ ]:
losses

In [ ]:
# 테스트 데이터셋
# 모델 불러올 경로 설정
load_path = f'save/{model_name}_epoch{num_epochs}_quntize({quantize}.pth)'

# 모델 및 다른 정보 불러오기
checkpoint = torch.load(load_path)
model.load_state_dict(checkpoint['model_state_dict'])  # 모델 가중치 불러오기

# 옵티마이저 상태 불러오기 (필요한 경우)
if 'optimizer_state_dict' in checkpoint:
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# 학습 에폭 불러오기 (선택적)
if 'epoch' in checkpoint:
    epoch = checkpoint['epoch']

# 모델을 평가 모드로 설정 (모델 불러온 후 필요한 경우)
model.eval()

print(f'Model loaded from {load_path}')
model.eval()

correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in tqdm_notebook(test_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy}')

In [ ]:
model_pretrained

In [ ]:
model_scratch, input_size_scratch = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained, quantize=quantize)
model_scratch = model_scratch.to(device)

model_scratch, hist_scratch = train_models(model_scratch, dataloaders_dict, criterion, optimizer, num_epochs=1, is_inception=False)

In [ ]:
from thop import profile

# 입력 크기 정의 (예: 3 채널, 224x224 크기의 이미지)
input_size = (128, 3, 224, 224)  # (배치 크기, 채널 수, 높이, 너비)

# 모델의 FLOPs 계산
macs, params = profile(model_scratch, inputs=(torch.randn(*input_size).to(device),))
print("FLOPs: {:.4f} G FLOPs".format(macs / 1e9))  # 결과를 기가 FLOPs로 표시합니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method and
#  the model trained from scratch
ohist = []
shist = []
num_epochs=1
ohist = [h.cpu().numpy() for h in hist_pretrained]
shist = [h.cpu().numpy() for h in hist_scratch]

plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
plt.plot(range(1,num_epochs+1),shist,label="Scratch")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.show()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

model_pretrained2, hist_pretrained2 = train_models(model_pretrained, dataloaders_dict, criterion, optimizer, num_epochs=1, is_inception=False)
model_scratch2, hist_scratch2 = train_models(model_scratch, dataloaders_dict, criterion, optimizer, num_epochs=1, is_inception=False)

In [ ]:
# # 모델 인스턴스 생성하고 구조 확인하기
# # 1. 일반 레즈넷, 마지막 fc레이어만 변경
# model_name = 'resnet'
# num_classes = 500
# feature_extract = False
# use_pretrained = True
# quantize = False
#
# model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True, quantize=False)
# print(model_name)
# print(model_ft)

In [ ]:
# # 2. 양자화된 레즈넷, 마지막 fc레이어만 변경 - QuantizedLinear 나와야하는데 일반 Linear만나옴
# model_name = 'resnet'
# num_classes = 100
# feature_extract = False
# use_pretrained = True
# quantize = True
#
# model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True, quantize=True)
# print('quantized ' + model_name)
# print(model_ft)

In [ ]:
# # 헬퍼함수 없이 만든 양자화된 레즈넷
# weights = models.quantization.ResNet50_QuantizedWeights.DEFAULT
# model_ft = models.quantization.resnet50(weights=weights, quantize=True)
# print(model_ft)

In [ ]:
# weights = models.quantization.MobileNet_V3_Large_QuantizedWeights.DEFAULT
# model_ft = models.quantization.mobilenet_v3_large(weigts=weights, quantize=True)
# print(model)